In [21]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import torch
import torch.nn as nn
import torch.utils.data as Data
from torch.utils.data import RandomSampler
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F
from sklearn.metrics import mean_squared_error
import pennylane as qml
import pytorch_lightning as pl
import math
from Factory import Window_that_slides, QShallowRegressionLSTM, QRegressionLSTM


In [22]:
dfi = pd.read_csv('./kaggle/city_hour.csv')
dfi

,City,Datetime,PM2.5,PM10,NO,NO2,NOx,NH3,CO,SO2,O3,Benzene,Toluene,Xylene,AQI,AQI_Bucket
0,Ahmedabad,2015-01-01 01:00:00,NaN,NaN,1.00,40.01,36.37,NaN,1.00,122.07,NaN,0.0,0.0,0.0,NaN,NaN
1,Ahmedabad,2015-01-01 02:00:00,NaN,NaN,0.02,27.75,19.73,NaN,0.02,85.90,NaN,0.0,0.0,0.0,NaN,NaN
2,Ahmedabad,2015-01-01 03:00:00,NaN,NaN,0.08,19.32,11.08,NaN,0.08,52.83,NaN,0.0,0.0,0.0,NaN,NaN
3,Ahmedabad,2015-01-01 04:00:00,NaN,NaN,0.30,16.45,9.20,NaN,0.30,39.53,153.58,0.0,0.0,0.0,NaN,NaN
4,Ahmedabad,2015-01-01 05:00:00,NaN,NaN,0.12,14.90,7.85,NaN,0.12,32.63,NaN,0.0,0.0,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
707870,Visakhapatnam,2020-06-30 20:00:00,9.50,36.00,2.75,25.57,15.85,4.57,0.62,NaN,27.75,NaN,NaN,NaN,51.0,Satisfactory
707871,Visakhapatnam,2020-06-30 21:00:00,17.25,49.25,3.62,33.20,20.62,3.78,0.76,2.02,25.58,NaN,NaN,NaN,51.0,Satisfactory
707872,Visakhapatnam,2020-06-30 22:00:00,36.00,71.00,2.20,30.80,18.20,3.67,0.58,1.77,26.15,NaN,NaN,NaN,50.0,Good
707873,Visakhapatnam,2020-06-30 23:00:00,15.75,63.00,1.02,28.90,16.00,3.80,0.49,0.75,15.82,NaN,NaN,NaN,50.0,Good


In [23]:
dfi.dropna()
df = dfi.loc[ dfi['City'] == 'Delhi']  #will be using the delhi data first for simplicity
df = df.loc[df['Datetime'] < '2016-01-01 00:00:00']
df = df.dropna()

In [24]:
df['Datetime'] = pd.to_datetime(df['Datetime'])
df = df.drop(['City','AQI_Bucket','Datetime'], axis=1)
df['nxtd_AQI'] = df['AQI'].shift(-1)
df['nxtd_AQI'] = df['AQI'].ffill(limit=200)
df

,PM2.5,PM10,NO,NO2,NOx,NH3,CO,SO2,O3,Benzene,Toluene,Xylene,AQI,nxtd_AQI
245187,211.51,340.66,13.09,37.08,47.07,33.54,15.24,12.33,34.35,3.51,5.85,4.73,456.0,456.0
245188,191.12,257.40,15.44,38.40,51.24,34.98,13.11,18.00,34.38,3.58,5.23,3.78,450.0,450.0
245189,218.44,351.01,22.85,44.73,59.26,34.80,18.35,19.17,38.42,3.67,5.93,4.98,446.0,446.0
245190,296.80,600.95,69.20,47.76,101.68,34.25,16.67,21.50,49.12,14.75,27.13,8.78,475.0,475.0
245191,336.43,714.63,148.42,46.31,171.10,35.75,12.17,17.67,56.44,26.56,45.62,9.99,480.0,480.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
253926,142.61,359.15,75.98,93.07,171.46,61.63,2.64,19.93,48.90,3.49,11.11,0.00,412.0,412.0
253927,201.88,464.06,155.64,107.08,109.14,72.92,3.79,23.26,79.43,5.37,16.95,0.00,415.0,415.0
253928,272.73,535.32,154.26,107.86,170.23,78.44,2.89,22.78,133.28,7.53,29.56,0.00,432.0,432.0
253929,271.46,518.95,134.38,99.07,179.56,82.17,3.02,24.44,110.69,7.39,28.62,0.02,433.0,433.0


In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4949 entries, 245187 to 253930
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   PM2.5     4949 non-null   float64
 1   PM10      4949 non-null   float64
 2   NO        4949 non-null   float64
 3   NO2       4949 non-null   float64
 4   NOx       4949 non-null   float64
 5   NH3       4949 non-null   float64
 6   CO        4949 non-null   float64
 7   SO2       4949 non-null   float64
 8   O3        4949 non-null   float64
 9   Benzene   4949 non-null   float64
 10  Toluene   4949 non-null   float64
 11  Xylene    4949 non-null   float64
 12  AQI       4949 non-null   float64
 13  nxtd_AQI  4949 non-null   float64
dtypes: float64(14)
memory usage: 580.0 KB


In [26]:
size = int(len(df) * 0.42)

df_train = df.iloc[:size].copy()
df_test = df.iloc[size:].copy()
df_train

,PM2.5,PM10,NO,NO2,NOx,NH3,CO,SO2,O3,Benzene,Toluene,Xylene,AQI,nxtd_AQI
245187,211.51,340.66,13.09,37.08,47.07,33.54,15.24,12.33,34.35,3.51,5.85,4.73,456.0,456.0
245188,191.12,257.40,15.44,38.40,51.24,34.98,13.11,18.00,34.38,3.58,5.23,3.78,450.0,450.0
245189,218.44,351.01,22.85,44.73,59.26,34.80,18.35,19.17,38.42,3.67,5.93,4.98,446.0,446.0
245190,296.80,600.95,69.20,47.76,101.68,34.25,16.67,21.50,49.12,14.75,27.13,8.78,475.0,475.0
245191,336.43,714.63,148.42,46.31,171.10,35.75,12.17,17.67,56.44,26.56,45.62,9.99,480.0,480.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247278,65.20,177.02,7.82,22.22,25.24,28.57,9.77,8.21,35.30,2.05,4.61,2.34,320.0,320.0
247279,59.05,101.48,7.73,23.36,25.61,34.23,9.84,11.46,38.30,1.93,4.46,1.86,314.0,314.0
247280,62.86,92.67,7.49,27.63,24.94,31.40,10.29,10.65,28.35,1.91,4.26,1.81,318.0,318.0
247281,88.09,182.08,8.08,31.38,24.43,29.85,9.18,12.35,24.68,2.04,4.69,2.42,316.0,316.0


In [27]:
df_train.columns[1:]

Index(['PM10', 'NO', 'NO2', 'NOx', 'NH3', 'CO', 'SO2', 'O3', 'Benzene',
       'Toluene', 'Xylene', 'AQI', 'nxtd_AQI'],
      dtype='object')

In [28]:
target = 'nxtd_AQI'
target_mean = df_train[target].mean()
target_stdev = df_train[target].std()

for c in df_train.columns:
    mean = df_train[c].mean()
    stdev = df_train[c].std()

    df_train[c] = (df_train[c] - mean) / stdev
    df_test[c] = (df_test[c] - mean) / stdev

In [29]:
features = ['PM2.5', 'PM10', 'NO', 'NO2', 'NOx', 'NH3', 'CO', 'SO2', 'O3','Benzene', 'Toluene', 'Xylene']

In [30]:
batch_size = 1
sequence_length = 3

train_dataset = Window_that_slides(
    df_train,
    target=target,
    features=features,
    sequence_length=sequence_length
)
test_dataset = Window_that_slides(
    df_test,
    target=target,
    features=features,
    sequence_length=sequence_length
)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

X, y = next(iter(train_loader))

print("Features shape:", X.shape)
print("Target shape:", y.shape)

Features shape: torch.Size([1, 3, 12])
Target shape: torch.Size([1])


In [31]:
def train_model(data_loader, model, loss_function, optimizer):
    num_batches = len(data_loader)
    total_loss = 0
    model.train()
    
    for X, y in data_loader:
        output = model(X)
        loss = loss_function(output, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / num_batches
    print(f"Train loss: {avg_loss}")
    return avg_loss



In [32]:
def test_model(data_loader, model, loss_function):
    
    num_batches = len(data_loader)
    total_loss = 0

    model.eval()
    with torch.no_grad():
        for X, y in data_loader:
            output = model(X)
            total_loss += loss_function(output, y).item()

    avg_loss = total_loss / num_batches
    print(f"Test loss: {avg_loss}")
    return avg_loss

In [33]:
def predict(data_loader, model):
    output = torch.tensor([])
    model.eval()
    with torch.no_grad():
        for X, _ in data_loader:
            y_star = model(X)
            output = torch.cat((output, y_star), 0)
    
    return output

In [34]:
len(features)

12

In [35]:
qml.version()

'0.34.0'

In [36]:
learning_rate = 0.33
num_hidden_units = 10

#num_sensors = 12

Qmodel = QShallowRegressionLSTM(num_sensors=len(features), hidden_units=num_hidden_units, n_qubits=4)
#Qmodel = QRegressionLSTM(num_sensors=len(features), hidden_units=num_hidden_units, n_qubits=4)
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(Qmodel.parameters(), lr=learning_rate)

weight_shapes = (n_qlayers, n_vrotations, n_qubits) = (1, 3, 4)


In [37]:
for i in range(4):
    print(i)

0
1
2
3


In [38]:
feature = tensor([-0.0257, -0.2504, -0.0076,  0.5170], grad_fn=<UnbindBackward0>)
ry_params = [tensor([-0.0257, -0.2453, -0.0076,  0.4771], grad_fn=<AtanBackward0>)]
rz_params = [tensor([0.0186, 0.1290, 0.0570, 0.0098], grad_fn=<AtanBackward0>)]


SyntaxError: expression cannot contain assignment, perhaps you meant "=="? (778710768.py, line 1)

In [ ]:
use_cuda = 1
device = torch.device("cuda" if (torch.cuda.is_available() & use_cuda) else "cpu")
for x, y in train_loader:
    x = x.to(device)
    y = y.to(device)
    output = Qmodel(X)
    print(output)

IndexError: list index out of range

In [ ]:
quantum_loss_train = []

for ix_epoch in range(13):
    print(f"Epoch {ix_epoch}\n---------")
    
    train_loss = train_model(train_loader, Qmodel, loss_function, optimizer=optimizer)

    quantum_loss_train.append(train_loss)



Epoch 0
---------


IndexError: list index out of range